## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [1]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [2]:
# Always remember to do this!
load_dotenv(override=True)

True

In [3]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [4]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [5]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [7]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


You are an independent advisor to the newly elected government of a fictional coastal city‑state of 1.2 million people with an annual public budget of $3 billion (projected to shrink by 0.5% per year) and GDP of $36 billion. Over the next decade the city faces five simultaneous challenges: accelerating sea‑level rise that threatens 20% of developed land within 30 years, a recurring contagious respiratory disease with periodic outbreaks, rising inequality and unemployment, an influx of 150,000 climate migrants over five years, and constrained fiscal capacity. Using plausible assumptions, produce the following:

1) A prioritized 10‑year plan that allocates the annual budget (give dollar amounts and percent of budget each year) across infrastructure (including coastal defenses and managed retreat), public health (prevention, surveillance, surge capacity), social programs (housing, basic income/training), economic stimulus (jobs, business support), and migration support (housing, integrati

In [8]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

## Note - update since the videos

I've updated the model names to use the latest models below, like GPT 5 and Claude Sonnet 4.5. It's worth noting that these models can be quite slow - like 1-2 minutes - but they do a great job! Feel free to switch them for faster models if you'd prefer, like the ones I use in the video.

In [9]:
# The API we know well
# I've updated this with the latest model, but it can take some time because it likes to think!
# Replace the model with gpt-4.1-mini if you'd prefer not to wait 1-2 mins

model_name = "gpt-5-nano"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Yes. Below is a structured, explicit plan using plausible assumptions and transparent calculations. All figures are in constant 2024 U.S. dollars. Assumptions are stated clearly at the start; where needed I flag ranges and key uncertainties.

Assumptions and context
- City: population 1.2 million; GDP ≈ $36 billion; annual city budget ≈ $3.0 billion in year 1, shrinking 0.5% per year in real terms (budget path: Year1 $3.000B; Year2 $2.985B; Year3 $2.970B; Year4 $2.955B; Year5 $2.940B; Year6 $2.925B; Year7 $2.910B; Year8 $2.896B; Year9 $2.881B; Year10 $2.867B).
- Risk framing:
  - Climate: accelerating sea-level rise endangers 20% of developed land within ~30 years; need coastal defenses and potential managed retreat.
  - Health: recurring contagious respiratory disease with outbreaks; requires prevention, surveillance, surge capacity.
  - Inequality/unemployment: rising gaps and joblessness; migrant inflow of 150k people over five years (average ~30k/year; uneven distribution).
  - Fiscal constraints: capacity is tight; debt/deficit not specified, but plan assumes no large new unfunded mandates; cost-sharing and phased investments favored.
- Discounting/valuation: for qualitative benefits I provide orders-of-magnitude (e.g., lives saved, avoided losses) using plausible ranges; where I quantify, I note the basis (e.g., VSL in the low-to-mid tens of millions, avoided damages as % of GDP etc.). The focus is on transparent tradeoffs and planning realism.

1) Prioritized 10-year plan (annual budget allocations by category)
Categories (definitions)
- Infrastructure (I): coastal defenses (seawalls, surge barriers, tidal gates), natural buffers (blue/green corridors), drainage/stormwater, and managed retreat actions (buyouts, land-use conversion, relocation support).
- Public Health (PH): prevention (vaccination, air quality, vector control, nutrition), surveillance (disease intel, wastewater monitoring, lab capacity), surge capacity (temporary clinics, staffing, stockpiles).
- Social Programs (SP): housing, basic income or guaranteed income-like stipend, training and re-skilling, social supports with emphasis on vulnerable cohorts.
- Economic Stimulus (ES): public works, job guarantees for critical services, small-business support, procurement to local firms, incubators for coastal/green tech.
- Migration Support (MS): housing for migrants, integration services (language, civic education), access to health/education, targeted social services.

Year-by-year allocations (dollars; percent of yearly budget)
Note: I apply the same category shares each year (Infrastructure 38%, Public Health 22%, Social Programs 20%, Economic Stimulus 12%, Migration Support 8%), which maps to roughly the plan described. This yields the following year-by-year allocations. All figures are approximate to the nearest million.

Year 1 (Budget $3.000B)
- Infrastructure: $1.14B (38%)
- Public Health: $0.66B (22%)
- Social Programs: $0.60B (20%)
- Economic Stimulus: $0.36B (12%)
- Migration Support: $0.24B (8%)

Year 2 (Budget $2.985B)
- Infrastructure: $1.134B
- Public Health: $0.656B
- Social Programs: $0.597B
- Economic Stimulus: $0.358B
- Migration Support: $0.239B

Year 3 (Budget $2.970B)
- Infrastructure: $1.129B
- Public Health: $0.653B
- Social Programs: $0.594B
- Economic Stimulus: $0.356B
- Migration Support: $0.237B

Year 4 (Budget $2.955B)
- Infrastructure: $1.122B
- Public Health: $0.650B
- Social Programs: $0.591B
- Economic Stimulus: $0.354B
- Migration Support: $0.236B

Year 5 (Budget $2.940B)
- Infrastructure: $1.117B
- Public Health: $0.646B
- Social Programs: $0.588B
- Economic Stimulus: $0.353B
- Migration Support: $0.235B

Year 6 (Budget $2.925B)
- Infrastructure: $1.111B
- Public Health: $0.643B
- Social Programs: $0.585B
- Economic Stimulus: $0.351B
- Migration Support: $0.234B

Year 7 (Budget $2.910B)
- Infrastructure: $1.106B
- Public Health: $0.640B
- Social Programs: $0.582B
- Economic Stimulus: $0.349B
- Migration Support: $0.232B

Year 8 (Budget $2.896B)
- Infrastructure: $1.100B
- Public Health: $0.637B
- Social Programs: $0.579B
- Economic Stimulus: $0.347B
- Migration Support: $0.231B

Year 9 (Budget $2.881B)
- Infrastructure: $1.095B
- Public Health: $0.634B
- Social Programs: $0.576B
- Economic Stimulus: $0.346B
- Migration Support: $0.230B

Year 10 (Budget $2.867B)
- Infrastructure: $1.089B
- Public Health: $0.631B
- Social Programs: $0.573B
- Economic Stimulus: $0.344B
- Migration Support: $0.229B

What these allocations aim to achieve (high level justification and expected benefits)
- Infrastructure (I)
  - Why: Address 20% land risk within ~30 years; coastal defenses reduce flood/erosion losses and enable more orderly managed retreat. A high upfront, lower-ongoing-risk spend pays off in avoided damages, maintains land-value and tax base, and preserves housing stock.
  - Quantified benefits and risks:
    - Benefits: Estimated PV of avoided damages from flooding and storm surge over 30 years in the high-risk area: roughly $14–20B (order-of-magnitude estimate; PV depends on discount rate and sea-level trajectory). By year 10, significant portion of defenses can be in service, preventing large outage costs and enabling land-use decisions.
    - Jobs: Tens of thousands of construction- and long-term maintenance-related jobs across the decade (roughly 12k–15k direct/contractor roles in peak build years; ongoing maintenance ~2k–3k/year).
    - Risks: Cost overrun, permitting delays, ecological side-effects, community pushback on retreat; uncertain time-to-complete defense system; risk of stranded assets if sea defenses outpace risk.
    - Opportunity costs: Fewer funds for immediate social needs in early years; risk that if climate scenarios are milder, some earlier defense spend outpaces actual risk.
- Public Health (PH)
  - Why: Recurrent outbreaks demand robust prevention, surveillance, and surge capacity to reduce mortality, hospital overload, and economic disruption; a healthier population yields higher productivity and lower long-term healthcare costs.
  - Benefits and risks:
    - Lives saved: estimated hundreds to low thousands over the decade depending on outbreak severity and vaccine/ventilation uptake; use of vaccination and rapid response could reduce fatalities by 20–60% per outbreak relative to baseline.
    - Economic: reduced productivity losses and lower emergency health expenditure; value of statistical life (VSL) used in policy assessment would place a high value on mortality reductions; more resilient workforce.
    - Risks: disease pathogen evolution, vaccine hesitancy, supply chain disruptions; surge capacity must be sustainable and avoid ghost payrolls.
    - Opportunity costs: Could crowd out longer-term investments in housing/training if oversized relative to actual outbreak burden; however, given recurrent outbreaks, a robust PH program is prudent.
- Social Programs (SP)
  - Why: Address inequality and unemployment pressures, reduce poverty traps, and support vulnerable residents and new migrants through housing subsidies and training.
  - Benefits and risks:
    - Jobs/skills: Training and apprenticeship components build human capital; targeted subsidies can reduce poverty and crime, improve housing stability.
    - Housing stability: Direct effects on homelessness, school outcomes, health; reduces long-term public costs.
    - Risks: Program leakage, dependency concerns if poorly designed; cost-control challenges with universal-ish guarantees; need to calibrate basic income/training to avoid disincentives.
    - Opportunity costs: Some resources redirected from infrastructure or PH; but social programs are critical to social compact and political stability.
- Economic Stimulus (ES)
  - Why: Direct job creation and firm support counter cyclical downturns, anchor local economies, and build capabilites that support infrastructure and migration integration.
  - Benefits and risks:
    - Jobs: Direct public works and business support create or preserve tens of thousands of jobs across the decade; multiplier effects on local economy.
    - Growth: Stimulus sustains GDP growth and tax receipts, easing fiscal pressure in later years.
    - Risks: Slippage to low-productivity projects; crowding-out of private investment; risk of inefficiency if procurement is not well-targeted.
    - Opportunity costs: If funds are skewed too far toward ES, long-run resilience investments (I) or PH may be underfunded.
- Migration Support (MS)
  - Why: A climate migrant wave requires housing, integration services, and access to services to prevent urban strain, social tensions, and unequal outcomes.
  - Benefits and risks:
    - Social stability and inclusion: Faster integration reduces dislocation, supports civic cohesion, and improves labor market absorption.
    - Housing and service costs: Initial costs but pro-growth as migrants contribute to economy and tax base.
    - Risks: Demand for housing spikes faster than supply; political sensitivity around migrants; must guard against discrimination and ensure equal access to services.
    - Opportunity costs: MS investments could be sensitive to migrant inflow pace; need clear integration metrics to avoid resource waste.

Summary of key strengths and tradeoffs in the 10-year plan
- Strengths: Early high investment in infrastructure reduces long-term damages; parallel public health build reduces outbreak costs; social programs and migration support address equity and labor supply; economic stimulus sustains jobs and private investment.
- Tradeoffs: Large upfront infrastructure spend reduces capability for other programs in Year 1; social programs require careful monitoring to avoid fiscal drift; migration support must be paired with localization and skill-building to yield durable labor-market gains.
- Overall expected returns: Net positive in long run if climate, health, and migration dynamics align with plan timing; ~10–20% of annual budgets directed at high-impact resilience and inclusive growth in the first 5 years, tapering into maintenance and optimization in later years.

2) Three biggest uncertainties, monitoring, triggers, and contingencies
Uncertainty A: Climate/sea-level projections and effectiveness of defenses
- What could go wrong: Sea-level rise worse than expected or defenses underperform; risk of accelerated inundation in the next 10–20 years; cost overruns or ecological/legal constraints delay implementation.
- Monitoring:
  - Annual coastal risk dashboards; monitor tide/sea-level data, storm surge frequency, land subsidence rates, and defense performance.
  - Independent engineering reviews every 2–3 years; track cost-to-completion versus budget.
  - Ecological impact monitoring (coastal ecosystems, sediment dynamics) to detect unintended consequences.
- Contingent triggers and plan adaptation:
  - Trigger: If modeled 50-year risk grows by >15% annually or per-year expected losses exceed baseline by >25% after two cycles, escalate defenses and consider accelerated retreat options.
  - Adaptation: Rebalance funds toward higher-emission scenario defenses, accelerate natural buffers, adjust managed retreat pace, and potentially accelerate revenue-generating or land-use changes in affected zones.

Uncertainty B: Magnitude and pace of migration inflow and integration outcomes
- What could go wrong: 150k climate migrants over five years is larger or slower than anticipated; housing stock and social services capacity are inadequate; integration outcomes (education, language, employment) lag, generating social strain.
- Monitoring:
  - Migrant housing occupancy, waiting times for housing, and service access metrics; track labor-market participation by migrant cohorts; language and skills accreditation rates.
  - Social cohesion indicators (crime, complaints, trust surveys) and school readiness outcomes.
  - Time to achieve target integration milestones (employment, education completion, stable housing) per cohort.
- Contingent triggers and plan adaptation:
  - Trigger: If housing demand exceeds supply by >20% for consecutive 6 months or migrant unemployment exceeds 12% in cohorts, adjust plan.
  - Adaptation: Increase MS and SP in years 2–5 (temporary housing subsidies, accelerated language/training programs, private-public housing partnerships, or renegotiate migrant intake pacing). Augment ES with targeted job-creation incentives for sectors with migrant labor demand.

Uncertainty C: Economic and fiscal capacity (growth vs. budget compression)
- What could go wrong: GDP growth slows, tax capacity underperforms, or debt service costs erode fiscal space; budget decline accelerates.
- Monitoring:
  - Quarterly economic indicators (growth, unemployment, poverty, wage growth), tax revenue trends, and debt metrics; monitor grant/aid exposure and contingent liabilities.
  - Public finance indicators: breakeven revenue thresholds, debt-to-GDP ratios, and liquidity buffers; track cost overruns in high-visibility projects.
- Contingent triggers and plan adaptation:
  - Trigger: If year-over-year revenue growth is <1.5% or debt service ratio worsens by more than 0.5 percentage points relative to plan for two consecutive years, pause or re-prioritize spending.
  - Adaptation: Temporarily slow or re-prioritize ES and SP spending; scale PH and I investments to essential minimums; broaden revenue mechanisms (public-private partnerships, concessional financing, climate funds) where politically feasible.

3) Three alternative 10-year strategies (A, B, C)
A) Maximize total welfare (utilitarian)
- Core changes to budget emphasis:
  - Infrastructure: ~42% (increasing defenses and retreat capacity to maximize avoided losses).
  - Public Health: ~24% (strong emphasis on prevention, surveillance, surge).
  - Social Programs: ~26% (broaden housing subsidies and universal or near-universal basic training with stipend to reduce inequality).
  - Economic Stimulus: ~8% (targeted but efficient job-creation, prioritize high-multiplier projects).
  - Migration Support: ~0% to 0%? In this plan, perhaps 0–4% depending on needs; but given a utilitarian lens, if migrants are essential to welfare, allocate 6–8%? To keep pure utilitarian, we would include Migration Support as small but sufficient to achieve social utility: 6% of budget in early years, tapering.
- Net effect: Maximize total welfare by reducing disease outbreaks, protecting land, and reducing inequality; higher upfront investment in SP and MS, with aggressive I/S adjustments to ensure long-term productivity.
- Ethical trade-offs: Potentially more coercive or redistributive policies; higher tax/regulation or financial commitments; but aims to maximize overall well-being.

B) Maximize individual freedoms and minimal coercion (libertarian/classical liberal)
- Core changes:
  - Infrastructure: Reduced footprint on mandatory relocation; emphasize private sector-led coastal protection and market-driven adaptation; increase use of public-private partnerships and private insurance mechanisms.
  - Public Health: Emphasize voluntary programs; minimize compulsory surges; rely on private sector innovations, testing, and voluntary compliance measures; maintain strong data privacy.
  - Social Programs: Narrow/means-tested SP; avoid universal basic income; ensure local participation in decisions and avoid coercive mandates; prioritize individual mobility and secondary markets.
  - Economic Stimulus: Expand private-sector-led job creation; reduce public project mandates; focus on enabling conditions (regulatory relief, business-financing ecosystems).
  - Migration Support: Emphasize voluntary integration measures; minimize coercive relocation; empower local communities to design integration programs; support language and job training but avoid universal mandates.
- Budget allocations: Infrastructure may dip to about 35–38%; SP remains around 18–22%; PH around 20–24% with a focus on voluntary compliance; MS modest at 6–8%; ES around 12–15%.
- Ethical trade-offs: Greater individual choice, potentially weaker safety nets for the worst-off; risk of higher social strain during rapid adaptation; more reliance on market mechanisms and community-based solutions.

C) Prioritize the worst-off (egalitarian)
- Core changes:
  - Infrastructure: Maintain strong defenses but ensure managed retreat and land-use changes are designed to minimize harm to low-income residents; increase public protections for the most vulnerable (e.g., subsidized housing in safe zones, targeted buyouts with universal protections).
  - Public Health: Large expansion for prevention, surveillance, and surge capacity; universal access to care; health equity focus; robust data collection on vulnerable groups.
  - Social Programs: Heavier emphasis on housing subsidies, universal basic income-like minimum supports, and broad training programs; more generous stipends; universal access to essential services.
  - Economic Stimulus: Allocate more to capacity-building in high-poverty areas; prioritize employment programs that target the poorest and most marginalized.
  - Migration Support: Stronger support for migrants to ensure access to housing, health, education; policies designed to prevent social exclusion; ensure that integration benefits are shared by host communities.
- Budget allocations: Infrastructure ~36–40%; PH ~26–28%; SP ~28–30% (highly generous); ES ~6–10%; MS ~6–8% (robust integration).
- Ethical trade-offs: Higher overall program cost; longer time to fiscal balance; potential burdens on taxpayers; requires strong political will and public communication to maintain legitimacy.

Notes on how allocations shift under each scenario
- A (utilitarian): front-load I and SP more heavily, with substantial MS in early years to maximize welfare gains for all; moderate MS in total.
- B (libertarian): minimize coercive spending; rely on markets; lower SP, MS, and some PH measures; prioritize enabling environment for private investment and private risk-sharing.
- C (egalitarian): massively invest in SP and MS and PH, ensuring universal access and protections for the worst-off; I remains high but sustainability constraints require careful design.

Summary of the three variants (at-a-glance):
- A: I 42%, PH 24%, SP 26%, ES 8%, MS 0–6% (depending on need)
- B: I 35–38%, PH 20–24%, SP 18–22%, ES 12–15%, MS 6–8%
- C: I 36–40%, PH 26–28%, SP 28–30%, ES 6–10%, MS 6–8%

4) Five concrete, high-impact actions within the first 90 days (why each is critical)
1) Establish a Coastal Resilience Authority and a Multi-Agency Disaster Readiness Unit (MADRU)
- Why: Centralizes planning, accelerates approvals, and coordinates I, PH, and MS investments; ensures consistency in retreat planning, hazard zoning, and emergency response.
- Critical outcomes: Clear governance, faster procurement, consistent risk messaging, cross-sector synergy.

2) Mobilize a 100-day Coastal Defenses Design-and-Implementation Plan with a transparent procurement framework
- Why: Commences the largest single-scale resilience effort; reduces upfront uncertainty and signals credibility to residents, lenders, and insurers.
- Critical outcomes: Concrete timelines, cost controls, and early protections for high-value assets (hospitals, critical infrastructure).

3) Launch Universal Access to Essential Public Health Services (vaccines, preventive care, and disease surveillance)
- Why: Short-term health security and long-term cost containment; reduces risk of large outbreaks affecting economic activity.
- Critical outcomes: Rapid coverage rates, robust surveillance network, and surge-capability readiness.

4) Initiate a Migrant Housing Accelerator and Integration Corridor
- Why: Stabilizes migrant housing, accelerates integration, and prevents social strain; ensures newly arriving residents are productive members of the economy.
- Critical outcomes: Housing units in place, language/skill programs launched, and early economic participation for migrants.

5) Pass a 90-day Social Housing and Training Pilot with performance contracts
- Why: Signals commitment to the worst-off, tests voucher-based housing plus targeted training with measurable milestones.
- Critical outcomes: Early evidence on housing stability, employment outcomes, and cost-to-benefit signals for expansion or adjustment.

5) Three measurable metrics (targets) to evaluate progress at Year 5 and Year 10, and one short-term indicator (first 6 months)
- Year 5 metrics and targets
  - Unemployment rate: target ≤ 5.5% (from current baseline ~7–9% depending on data); monitoring: jobs created via ES and SP; track youth and long-term unemployed separately.
  - Housing stability among vulnerable residents: target 85% of eligible households with stable housing (down from higher baseline homelessness/eviction risk levels); monitor SP and MS housing subsidies; measure average months in stable housing.
  - Land area protected from coastal flooding/damage: target 40% of at-risk developed land protected by defenses or relocations; monitor coastal risk indices, insurance exposure, and asset value preserved.
- Year 10 metrics and targets
  - Gini coefficient or equivalent inequality index: target improvement of 0.03–0.05 (i.e., substantial narrowing of income inequality) relative to baseline; monitor income distribution and access to services.
  - GDP growth and resilience index: target sustained real GDP growth around 2.0–2.5% annually; monitor resilience to shocks (health, climate, migration); include unemployment and productivity metrics.
  - Share of migrants integrated into the formal economy (employment, education completion, language proficiency): target 60–70% of migrant cohorts integrated into local labor markets or education pathways; monitor by cohort and region.
- Short-term indicator (first 6 months)
  - Coastal defenses procurement progress and 100-day implementation milestones: percentage of critical defense contracts awarded and initial construction completions; ensure a visible start and momentum; track adherence to schedule.

6) Parts I am least confident about, data gaps, and what would improve confidence
- Greatest uncertainties:
  - Climate-risk trajectories and structural responses: precise timing of sea-level rise, storm surge paths, and performance of defenses; improved confidence requires finer-grained coastal-bank data, sedimentation models, and ongoing monitoring of risk-relevant parameters.
  - Migration integration outcomes: exact pace and economic contribution of migrants; need data on housing availability, language proficiency, credential recognition, and regional labor-market absorption.
  - Fiscal capacity and macroeconomic response: long-run GDP growth, tax-base expansion, and debt capacity; need more robust budgetary forecasting, debt service projections, and potential external financing scenarios.
- How to improve: establish a data warehouse with monthly updates across climate, health, migration, housing, and labor metrics; require regular independent reviews; incorporate scenario analysis with alternate policy knobs (e.g., higher/lower migration, faster/slower infrastructure build) to quantify sensitivity.

7) Executive summary (50 words)
- “We shield our city from rising seas, outbreaks, and inequality with a disciplined, phased plan: build defenses, fortify health, house migrants and the displaced, create jobs, and train residents. It costs today, but preserves tomorrow’s land, lives, and livelihoods—safeguarding prosperity for all.”

Key calculations and compact reasoning notes
- Budget trajectory: B_t = 3.000B × (0.995)^(t-1) for year t (t=1..10).
- Category shares: Infrastructure 38%, Public Health 22%, Social Programs 20%, Economic Stimulus 12%, Migration Support 8% (approximate and adjustable with plan risk).
- Yearly allocations derived by multiplying B_t by category shares; example Year 1:
  - I: 3.000 × 0.38 = 1.14B
  - PH: 3.000 × 0.22 = 0.66B
  - SP: 3.000 × 0.20 = 0.60B
  - ES: 3.000 × 0.12 = 0.36B
  - MS: 3.000 × 0.08 = 0.24B
- The plan emphasizes large, earlier investments in infrastructure and public health to reduce long-run expected damages and health losses, while social programs and migration support address equity and labor-market dynamics; economic stimulus funds are kept moderate to avoid crowding out essential resilience investments.

Would you like me to provide a compact, at-a-glance slide deck version (one slide per year with bullets and numbers), or a more detailed appendix with computations (PV of avoided losses, rough lives saved values, and simple ROI estimates) to aid briefing and procurement planning?

In [10]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-sonnet-4-5"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

# Strategic 10-Year Plan for Coastal City-State
## Advisory Report to the Mayor

---

## EXECUTIVE SUMMARY (50 words)
Front-load coastal defense and public health infrastructure now while budgets allow. Integrate climate migrants as economic contributors through housing-plus-employment programs. Shift from defensive spending to economic stimulus by year 4. Accept strategic retreat from 8% of threatened land. Success requires immediate action, adaptive monitoring, and treating migration as opportunity, not burden.

---

## 1. PRIORITIZED 10-YEAR BUDGET ALLOCATION PLAN

### Key Assumptions
- Annual budget starts at $3.0B, declines 0.5%/year → Year 10: $2.87B
- Current population: 1.2M + 150K migrants over 5 years (30K/year) = 1.35M by Year 5
- 20% of developed land = ~$7.2B in property value (20% of GDP as land/structures)
- Baseline disease outbreak: 2-3 per decade, each costing $120M in lost productivity + $40M direct health costs
- Unemployment baseline: 8%, target: 5.5% by Year 10

### Annual Budget Allocation (Years 1-10)

**YEARS 1-3: Crisis Stabilization & Infrastructure Investment**

| Category | Year 1 | Year 2 | Year 3 |
|----------|---------|---------|---------|
| **Coastal Defense & Managed Retreat** | $900M (30%) | $780M (26%) | $660M (22%) |
| **Public Health** | $450M (15%) | $420M (14%) | $390M (13%) |
| **Social Programs** | $540M (18%) | $570M (19%) | $600M (20%) |
| **Economic Stimulus** | $660M (22%) | $720M (24%) | $780M (26.3%) |
| **Migration Support** | $300M (10%) | $330M (11%) | $330M (11.1%) |
| **Core Services** | $150M (5%) | $165M (5.5%) | $165M (5.6%) |
| **TOTAL** | $3,000M | $2,985M | $2,970M |

**YEARS 4-7: Transition to Growth**

| Category | Year 4 | Year 5 | Year 6 | Year 7 |
|----------|---------|---------|---------|---------|
| **Coastal Defense** | $480M (16%) | $420M (14%) | $360M (12%) | $300M (10%) |
| **Public Health** | $360M (12%) | $360M (12%) | $360M (12%) | $330M (11%) |
| **Social Programs** | $630M (21.3%) | $660M (22.3%) | $630M (21.3%) | $600M (20.3%) |
| **Economic Stimulus** | $840M (28.4%) | $870M (29.4%) | $930M (31.5%) | $990M (33.5%) |
| **Migration Support** | $300M (10.1%) | $240M (8.1%) | $180M (6.1%) | $120M (4.1%) |
| **Core Services** | $345M (11.7%) | $405M (13.7%) | $495M (16.8%) | $615M (20.8%) |
| **TOTAL** | $2,955M | $2,955M | $2,955M | $2,955M |

**YEARS 8-10: Sustainable Operations**

| Category | Year 8 | Year 9 | Year 10 |
|----------|---------|---------|---------|
| **Coastal Defense** | $240M (8%) | $210M (7%) | $180M (6%) |
| **Public Health** | $330M (11%) | $300M (10%) | $300M (10%) |
| **Social Programs** | $570M (19%) | $540M (18.3%) | $510M (17.8%) |

In [11]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

As an independent advisor, I present the following 10-year strategic plan for your city-state. My recommendations are based on a balanced approach to the concurrent challenges, aiming for resilience and sustainable growth within the stated fiscal constraints.

**Core Assumptions:**

1.  **Inflation:** Negligible impact on real budget over 10 years for simplicity, or absorbed by efficiency gains.
2.  **External Aid:** No significant external aid beyond the public budget is assumed.
3.  **Technology:** Current technologies for coastal defense, public health, and urban planning are available and cost-effective.
4.  **Political Will:** Strong political will to implement difficult but necessary reforms and investments.
5.  **Disease:** The recurring respiratory disease has a vaccine/treatment but requires ongoing surveillance and occasional surge capacity.
6.  **Economic Multipliers:** Standard economic multipliers are assumed (e.g., infrastructure $1: $1.5-2.5 GDP; social programs $1: $1.2-1.8 GDP).
7.  **Cost Estimates (Illustrative):**
    *   Coastal defense: $5-10M per km, major projects requiring hundreds of millions.
    *   Managed retreat: Acquisition/relocation $500K-$1M per household.
    *   Public health: Prevention/surveillance $50-100 per capita annually; surge capacity $50-150M per major outbreak.
    *   Affordable housing: $150K-$300K per unit.
    *   Job training: $5K-$15K per person.
    *   Basic income: $500/month per person for target population.
    *   Migration support: Initial settlement/housing $5K-$10K per person.

---

### 1) Prioritized 10-Year Plan

**Annual Public Budget:**
*   Year 1: $3.00 Billion
*   Year 2: $2.985 Billion
*   Year 3: $2.970 Billion
*   Year 4: $2.955 Billion
*   Year 5: $2.940 Billion
*   Year 6: $2.926 Billion
*   Year 7: $2.911 Billion
*   Year 8: $2.896 Billion
*   Year 9: $2.882 Billion
*   Year 10: $2.867 Billion
*   **Total Budget over 10 years: ~$29.33 Billion**

**Prioritization Logic:**
The plan prioritizes immediate human safety (public health, migrant housing) and long-term existential threats (sea-level rise) while building foundational resilience through social and economic programs. A phased approach is critical given the shrinking budget.

| Category                      | Year 1-3 (Stabilization & Planning) | Year 4-7 (Implementation & Growth) | Year 8-10 (Sustained Resilience) | Average 10-Year Allocation | Justification, Benefits, Risks, Opportunity Costs |
| :---------------------------- | :---------------------------------- | :--------------------------------- | :------------------------------- | :------------------------- | :------------------------------------------------ |
| **Infrastructure**            | **25% ($750M/yr)**                  | **35% ($1.03B/yr)**                | **30% ($860M/yr)**               | **30% ($870M/yr)**         | **Justification:** Addressing existential SLR threat is non-negotiable. Early investment allows for phased implementation of defenses and managed retreat planning. |
| (Coastal Defenses, Managed Retreat, Critical Infrastructure upgrades) |                                     |                                    |                                  |                            | **Expected Benefits:** Avoided ~ $5-10B in property damage; protection of ~240,000 residents in vulnerable areas; sustained economic activity in protected zones; creates ~5,000-10,000 jobs. <br> **Main Risks:** Cost overruns, engineering failures, public resistance to managed retreat, accelerated SLR requiring re-design. <br> **Opportunity Costs:** Less immediate investment in social safety nets or direct economic stimulus, potentially slowing short-term growth or increasing inequality. |
| **Public Health**             | **20% ($600M/yr)**                  | **15% ($440M/yr)**                 | **10% ($287M/yr)**               | **15% ($437M/yr)**         | **Justification:** Immediate crisis; prevents widespread illness, economic disruption, and strain on healthcare. Early investment builds robust systems. |
| (Prevention, Surveillance, Surge Capacity, Vaccine Procurement) |                                     |                                    |                                  |                            | **Expected Benefits:** Saves ~500-1,000 lives annually (conservative); prevents ~ $1-2B in economic losses from outbreaks; reduces healthcare system burden; protects workforce productivity. <br> **Main Risks:** New disease variants, vaccine hesitancy, public non-compliance with health measures, global supply chain disruptions. <br> **Opportunity Costs:** Fewer resources for direct job creation or large-scale housing projects; some funds might sit idle if outbreaks are less severe than projected. |
| **Migration Support**         | **15% ($450M/yr)**                  | **10% ($295M/yr)**                 | **5% ($143M/yr)**                | **10% ($290M/yr)**         | **Justification:** 150,000 climate migrants require immediate, humane, and structured integration to prevent social unrest and leverage new labor. Front-loaded for initial intake. |
| (Housing, Integration Services, Initial Needs) |                                     |                                    |                                  |                            | **Expected Benefits:** Houses ~75,000-100,000 migrants within 5 years; prevents humanitarian crisis; harnesses new labor (potential 5% increase in labor force); contributes to GDP growth by ~$500M-$1B annually post-integration. <br> **Main Risks:** Social tensions, integration failures, higher-than-expected migrant influx, strain on existing services, inadequate job skills match. <br> **Opportunity Costs:** Less direct investment in existing citizen welfare programs or infrastructure; high upfront costs for uncertain long-term integration success. |
| **Social Programs**           | **20% ($600M/yr)**                  | **20% ($590M/yr)**                 | **25% ($718M/yr)**               | **22% ($640M/yr)**         | **Justification:** Rising inequality and unemployment fuel social instability. Investment in housing and skills is critical for long-term equity and productivity. |
| (Affordable Housing, Basic Income/Training, Education, Social Safety Nets) |                                     |                                    |                                  |                            | **Expected Benefits:** Creates ~10,000-15,000 affordable housing units; lifts ~50,000 people out of poverty; boosts labor force participation; reduces crime by ~10-15%; improves social cohesion. ROI up to $1.8 per $1 invested. <br> **Main Risks:** Program inefficiency, dependency on basic income, skills mismatch with job market, political pushback on welfare. <br> **Opportunity Costs:** Diverts funds from direct private sector stimulus or more immediate infrastructure projects, potentially slower economic recovery in the short term. |
| **Economic Stimulus**         | **20% ($600M/yr)**                  | **20% ($590M/yr)**                 | **30% ($860M/yr)**               | **23% ($670M/yr)**         | **Justification:** Essential for job creation, diversifying the economy, and providing tax revenue for public services. Long-term focus after initial stabilization. |
| (Job Creation, Business Support, R&D, Diversification Initiatives) |                                     |                                    |                                  |                            | **Expected Benefits:** Creates ~15,000-20,000 new jobs; boosts GDP by ~1-2% annually; increases tax revenue; fosters innovation and resilience against future shocks. <br> **Main Risks:** Ineffective targeting of funds, market failures, "picking winners" risks, global economic downturns, regulatory hurdles. <br> **Opportunity Costs:** Less immediate relief for vulnerable populations; some projects may not deliver expected returns; potential for cronyism. |
| **Total**                     | **100% ($3.0B/yr)**                 | **100% ($2.95B/yr)**               | **100% ($2.87B/yr)**             | **100% ($2.93B/yr)**       |                                                                   |

---

### 2) Three Biggest Uncertainties

1.  **Severity & Frequency of Respiratory Disease Outbreaks:**
    *   **Quantification:** Current plan assumes 1-2 moderate outbreaks over 10 years requiring surge capacity ($50-150M each). Uncertainty: Could be 3-5 severe outbreaks ($150-300M each, requiring up to $1.5B over 10 years, or 5-10% of total budget) or negligible if a highly effective, long-lasting vaccine is developed.
    *   **Monitoring:**
        *   **Indicator:** Real-time R0 (reproduction number) of disease, weekly hospitalizations/ICU admissions, vaccine uptake rates, genomic surveillance for new variants.
        *   **Body:** Public Health Agency, supported by genomic sequencing labs.
    *   **Contingent Triggers & Adaptation:**
        *   **Trigger 1 (Worsening):** R0 consistently above 1.5 for 3 weeks, combined with >75% ICU capacity utilization and significant workforce absenteeism (e.g., >5%).
        *   **Adaptation:** Immediately reallocate 5% of the Economic Stimulus budget (approx. $150M-$200M annually) to Public Health for emergency surge capacity, procure additional vaccines/therapeutics, and implement targeted lockdowns/mitigation measures. This would temporarily slow economic growth initiatives.
        *   **Trigger 2 (Improving):** R0 consistently below 0.8 for 6 months, <10% ICU capacity, and widespread (80%+) long-term immunity.
        *   **Adaptation:** Reallocate 5% of Public Health budget (approx. $150M) to Social Programs (affordable housing/training) or Economic Stimulus to address other pressing needs.

2.  **Rate and Extent of Sea-Level Rise (SLR) & Extreme Weather Events:**
    *   **Quantification:** Current plan assumes a moderate SLR trajectory and allows for proactive but staged defenses. Uncertainty: Could accelerate rapidly (e.g., 20% land lost in 15-20 years instead of 30), requiring double the infrastructure budget ($1.7B annually) or forcing more extensive and costly managed retreat (~$10-20B additional over 10 years).
    *   **Monitoring:**
        *   **Indicator:** Satellite altimetry data, local tide gauge measurements, frequency/intensity of storm surges, coastal erosion rates.
        *   **Body:** Environmental Agency, Meteorological Office, in partnership with international climate science bodies.
    *   **Contingent Triggers & Adaptation:**
        *   **Trigger 1 (Worsening):** Measured SLR exceeding 5mm/year consistently for 3 years, or 3 "100-year" flood events within a 5-year period.
        *   **Adaptation:** Increase Infrastructure budget by 5-10% (approx. $150M-$300M annually), primarily reallocating from Economic Stimulus and reducing Social Programs growth. Accelerate managed retreat planning and implementation for high-risk areas, potentially using eminent domain. Invest in rapid-deployment temporary defenses.
        *   **Trigger 2 (Improving):** SLR rate slows to <2mm/year for 5 years, with no significant increase in extreme weather events.
        *   **Adaptation:** Reduce Infrastructure allocation by 5% (approx. $150M annually), reallocating to Economic Stimulus for long-term diversification or increasing Social Programs. Shift focus to nature-based solutions.

3.  **Economic Performance (GDP Growth) & Fiscal Capacity:**
    *   **Quantification:** Plan assumes a projected 0.5% budget shrinkage, which is manageable. Uncertainty: A severe global recession or domestic economic shock could lead to a 2-3% annual budget shrinkage (up to $900M reduction over 10 years) or higher unemployment. Conversely, strong GDP growth could stabilize or even slightly increase the budget.
    *   **Monitoring:**
        *   **Indicator:** Quarterly GDP growth, unemployment rate, inflation, key industry performance indicators, tax revenue collection.
        *   **Body:** Ministry of Finance, National Statistics Office.
    *   **Contingent Triggers & Adaptation:**
        *   **Trigger 1 (Worsening):** Sustained negative GDP growth (>1% for 2 consecutive quarters), or actual tax revenue falling >5% below projections for 2 consecutive years.
        *   **Adaptation:** Implement an across-the-board 5-10% budget cut to all non-essential operational expenses. Prioritize core infrastructure and public health. Freeze hiring. Explore emergency revenue measures (e.g., temporary surtaxes on luxury goods, bond issuance if market allows). Reduce new social program rollouts.
        *   **Trigger 2 (Improving):** Sustained GDP growth >3% for 2 consecutive years, and tax revenue >5% above projections.
        *   **Adaptation:** Create a dedicated "Resilience Fund" to save surplus, or strategically increase allocations to areas with high ROI, such as R&D within Economic Stimulus or scaling up Social Programs with proven effectiveness. Consider small, targeted tax relief measures to stimulate further growth.

---

### 3) Three Alternative 10-Year Strategies

**A) Maximizing Total Welfare (Utilitarian Strategy)**

*   **Core Principle:** Maximize the greatest good for the greatest number. Decisions are based on aggregate cost-benefit analysis, even if it means some individuals or groups face disadvantages.
*   **Budget Allocation Shifts:**
    *   **Infrastructure:** 40% (+$10%): High investment in hard coastal defenses for maximum protection of developed land and economic assets. Prioritize projects with highest economic ROI (e.g., protecting CBD, industrial zones). Managed retreat is a last resort, focused on areas with lowest economic value.
    *   **Economic Stimulus:** 30% (+$7%): Aggressive investment in R&D, business incentives, and export-oriented industries to maximize GDP growth and create jobs, benefiting the largest segment of the population. Focus on high-skill sectors.
    *   **Public Health:** 15% (No change): Maintain robust prevention and surge capacity as widespread disease negatively impacts overall productivity and welfare.
    *   **Migration Support:** 10% (No change): Integrate migrants for their economic contribution and to avoid negative externalities (e.g., social instability) that reduce overall welfare. Focus on skills matching and quick employment.
    *   **Social Programs:** 5% (-17%): Minimal safety nets. Focus on programs with clear, measurable returns (e.g., targeted job training for in-demand skills). Affordable housing is largely left to market forces; basic income is deemed inefficient unless it directly promotes labor market re-entry.
*   **Key Actions:**
    *   Launch "Mega-Projects" for coastal protection of economic hubs.
    *   Aggressive tax breaks and incentives for high-growth industries.
    *   Streamline bureaucracy for businesses and skilled migrant workers.
    *   Public-private partnerships for essential services.
*   **Ethical Trade-offs:**
    *   **Gains:** Highest overall GDP growth, most infrastructure protection, potentially highest aggregate quality of life for the majority.
    *   **Losses:** Increased inequality, neglect of the most vulnerable (those in unprotected coastal areas, long-term unemployed, or those who cannot re-skill), potential for social unrest if wealth disparities become too extreme. Sacrifices individual rights/needs for collective benefit.

**B) Maximizing Individual Freedoms and Minimal Coercion (Libertarian/Classical Liberal Strategy)**

*   **Core Principle:** Emphasize individual liberty, property rights, free markets, and limited government intervention. Government's role is to protect rights, enforce contracts, and provide only essential public goods that cannot be supplied by the market.
*   **Budget Allocation Shifts:**
    *   **Economic Stimulus:** 35% (+$12%): Focus on deregulation, tax cuts, and creating a highly competitive business environment. Minimal direct subsidies. Government's role is to remove barriers to market flourishing.
    *   **Infrastructure:** 25% (No change): Focus on essential public goods (roads, ports, basic utilities). Coastal defenses are primarily left to private property owners or limited to critical national infrastructure. Managed retreat would be voluntary and market-based, with minimal government compensation beyond basic property rights.
    *   **Public Health:** 10% (-5%): Focus on providing information and education. Healthcare is primarily private; government provides a basic "safety net" for catastrophic illness or public health emergencies, but largely relies on individual responsibility and private insurance.
    *   **Migration Support:** 10% (No change): Welcome migrants but provide minimal direct state support, expecting them to be self-sufficient and integrate through market mechanisms (e.g., private housing, job finding). Focus on legal processing and protecting their rights.
    *   **Social Programs:** 20% (No change): Acknowledge a minimal safety net for absolute destitution (e.g., emergency shelters) but no broad basic income or large-scale affordable housing programs, as these interfere with market freedom and individual responsibility. Focus on charity encouragement.
*   **Key Actions:**
    *   Significant tax cuts and deregulation across all sectors.
    *   Privatization of some public services.
    *   Encourage private insurance for health and disaster relief.
    *   Strict enforcement of property rights.
*   **Ethical Trade-offs:**
    *   **Gains:** Maximizes individual choice and economic freedom, potentially fostering rapid innovation and dynamic markets. Minimal tax burden, attracting capital.
    *   **Losses:** High risk of exacerbated inequality, inadequate protection for the vulnerable (especially from disease and climate impacts), potential social instability, "tragedy of the commons" for public goods like coastal defense if not adequately addressed by market mechanisms. Those unable to help themselves are left behind.

**C) Prioritizing the Worst-Off (Egalitarian Strategy)**

*   **Core Principle:** Maximize the well-being of the least advantaged members of society. Resources are directed to uplift those at the bottom, reduce inequality, and ensure basic needs are met for all.
*   **Budget Allocation Shifts:**
    *   **Social Programs:** 35% (+$13%): Massive investment in affordable housing, universal basic income (or robust social safety nets), education, and job training for the unemployed and underemployed. Focus on eradicating poverty and ensuring access to essential services.
    *   **Migration Support:** 20% (+$10%): Provide comprehensive support for migrants, ensuring immediate housing, language training, job placement, and full integration into social services, recognizing them as a vulnerable group.
    *   **Public Health:** 20% (No change): Maintain strong universal public health services, ensuring equitable access to prevention, treatment, and surge capacity for all, regardless of income.
    *   **Infrastructure:** 15% (-15%): Prioritize protection/relocation of vulnerable communities over economically valuable but non-residential areas. Focus on nature-based solutions where possible to minimize cost. Managed retreat is planned and compensated generously for affected residents.
    *   **Economic Stimulus:** 10% (-13%): Shift focus from broad business support to social enterprises, worker cooperatives, and industries that create jobs for marginalized groups. Tax wealthy individuals/corporations to fund social programs.
*   **Key Actions:**
    *   Aggressive wealth redistribution policies (e.g., progressive taxation).
    *   Large-scale public housing projects and rent controls.
    *   Universal access to healthcare and education.
    *   Community-led managed retreat with full compensation and support for relocation.
*   **Ethical Trade-offs:**
    *   **Gains:** Significantly reduces poverty and inequality, strengthens social cohesion, provides a strong safety net for all. Ensures basic dignity and rights for the most vulnerable.
    *   **Losses:** Potentially slower overall economic growth due to higher taxes/regulation and reduced incentives for capital accumulation. Risk of dependency on state support. May struggle to fund large-scale, capital-intensive infrastructure like hard coastal defenses, potentially leaving more land vulnerable in the long run if only vulnerable communities are protected.

---

### 4) Five Concrete, High-Impact Actions (First 90 Days)

1.  **Establish a Unified "Climate-Health-Migration Resilience Task Force":** This inter-ministerial body will immediately begin a rapid **vulnerability assessment and action plan** for coastal areas, map disease hotspots/vulnerable populations, and coordinate migrant intake.
    *   **Why Critical:** Breaks down silos, ensures coordinated response, provides a holistic understanding of intertwined challenges, and prepares the groundwork for the 10-year plan.

2.  **Launch Emergency Climate Migrant Welcome & Integration Centers:** Allocate initial funds ($50M-$100M) for establishing temporary housing, registration, basic needs provision, and preliminary skills assessment for the immediate influx of migrants.
    *   **Why Critical:** Addresses the immediate humanitarian crisis, prevents social instability, and sets a positive tone for integration, crucial for leveraging new human capital rather than creating a burden.

3.  **Initiate a Comprehensive Fiscal Review & Budget Prioritization Consultation:** Engage experts and the public in a transparent review of current spending, revenue streams, and long-term fiscal projections. Use this to refine budget allocations and build public consensus.
    *   **Why Critical:** Builds trust, ensures fiscal discipline given the shrinking budget, identifies potential inefficiencies, and prepares the city for the necessary tough choices ahead.

4.  **Announce "Healthy City 2030" Public Health Preparedness Program:** Immediately audit existing healthcare surge capacity, procure essential medical supplies for future outbreaks, and launch a public awareness campaign on respiratory disease prevention.
    *   **Why Critical:** Proactive health measures save lives and prevent economic disruption. It reassures citizens, builds public health infrastructure, and demonstrates government's commitment to immediate safety.

5.  **Fast-Track "Green Jobs for the Future" Initiative:** Immediately identify 3-5 high-growth sectors (e.g., renewable energy, sustainable aquaculture, climate tech, care economy) and launch small-scale pilot programs for skills training and business support, specifically targeting unemployed youth and migrants.
    *   **Why Critical:** Begins to address unemployment and economic diversification from day one, creates hope, and links economic recovery to climate adaptation and integration.

---

### 5) Three Measurable Metrics and One Short-Term Indicator

**Long-Term Metrics (Year 5 & Year 10 Targets):**

1.  **Coastal Resilience & Safety:**
    *   **Target (Year 5):** 50% of critical infrastructure and 25% of the population in identified high-risk coastal areas are protected by new defenses or are part of managed retreat programs.
    *   **Target (Year 10):** 80% of critical infrastructure and 60% of the population in high-risk coastal areas are protected or have been safely relocated/compensated.
2.  **Socio-Economic Equity & Opportunity:**
    *   **Target (Year 5):** Unemployment rate reduced to <8%; Gini coefficient reduced by 5% from baseline; 50,000 individuals supported through social programs (housing/training).
    *   **Target (Year 10):** Unemployment rate <5%; Gini coefficient reduced by 10%; 100,000 individuals supported; poverty rate reduced by 25%.
3.  **Public Health & Disease Control:**
    *   **Target (Year 5):** Morbidity and mortality rates from recurring respiratory disease reduced by 30% from baseline (average of pre-plan outbreaks), and healthcare system surge capacity meets 150% of peak historical demand.
    *   **Target (Year 10):** Morbidity and mortality rates reduced by 50% from baseline, with healthcare system surge capacity meeting 200% of peak demand, and 90% vaccine uptake for targeted populations.

**Short-Term Indicator (First 6 Months):**

*   **Migrant Intake & Basic Needs Provision Rate:**
    *   **Indicator:** Percentage of new climate migrants (estimated 15,000 in first 6 months) successfully registered, temporarily housed, and provided with basic needs within 72 hours of arrival.
    *   **Target (6 months):** >90% of arriving migrants successfully processed and settled into temporary arrangements.

---

### 6) Least Confident Parts and Data/Analysis Needs

I am least confident about the **long-term social integration success of climate migrants and its reciprocal impact on social cohesion and public services.**

*   **Why:** This is a complex socio-cultural dynamic that is hard to model. While the plan allocates significant resources, successful integration depends on numerous variables beyond government control: host community attitudes, migrants' prior skills and adaptability, cultural differences, and unforeseen social frictions. There's a risk of creating parallel societies or increasing social polarization despite best efforts. Quantifying the precise economic benefit (or cost) of integration is also challenging, as it depends on whether migrants become net contributors or remain reliant on state support.

*   **Data Gaps/Hard-to-Model Dynamics:**
    *   **Behavioral Economics/Sociology:** The degree to which job training leads to employment for diverse groups, the effectiveness of integration programs in fostering community bonds, and the tipping points for social unrest versus successful multiculturalism.
    *   **Feedback Loops:** How migrant influx might affect local wages, housing prices, and public service demand for existing citizens, and how these changes then feed back into social acceptance of migrants.
    *   **Psychological Impacts:** The mental health burden on migrants and existing populations due to climate change, disease, and societal shifts.

*   **Additional Data or Analysis to Improve Confidence:**
    1.  **Longitudinal Cohort Study of Migrant Integration:** Track a sample of arriving migrants over 5-10 years, measuring employment, housing stability, language proficiency, social networks, and self-reported well-being.
    2.  **Public Opinion Surveys & Focus Groups:** Conduct regular, granular surveys with both existing residents and migrants to gauge social cohesion, perceptions of integration programs, and identify sources of friction or success.
    3.  **Economic Impact Modeling with Disaggregated Data:** Develop more sophisticated economic models that can disaggregate the impact of migration on specific labor markets, housing sectors, and public services, rather than just aggregate GDP. This requires granular data on skills, demographics, and regional economic activity.
    4.  **Case Studies & Best Practices:** Analyze successful and unsuccessful migrant integration programs from other global cities facing similar challenges, adapting lessons learned to the city-state's unique context.

---

### 7) Executive Summary (50 words or fewer)

This 10-year plan builds a resilient future by tackling sea-level rise and disease while integrating migrants and fostering economic opportunity. We front-load crucial investments, adapt to uncertainties, and track progress, ensuring safety, equity, and prosperity for all citizens within our shrinking budget.

In [12]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Yes, I can produce this plan. The following is a comprehensive 10-year strategic framework for the fictional coastal city-state.

### **Key Assumptions**

*   **Baseline Budget:** Starts at $3.0 billion in Year 1, shrinking by 0.5% annually (Year 2: $2.985B, Year 10: ~$2.87B).
*   **GDP:** $36 billion provides a tax base of ~8.3% of GDP to fund the $3B budget.
*   **Sea-Level Rise (SLR):** A 1-in-100-year flood event could cause >$5 billion in direct damage and economic disruption. Coastal defense is a capital-intensive, multi-year project.
*   **Disease:** A moderate outbreak costs ~$500M in healthcare and lost productivity; a severe pandemic could cost >$2B.
*   **Unemployment:** Baseline rate of 6%. Creating a job through public works costs ~$75,000/year (salary + materials).
*   **Climate Migrants:** 30,000/year for 5 years. Basic housing and integration support costs ~$5,000/person/year initially.
*   **Economic Multiplier:** Public infrastructure investment has a multiplier of 1.3x-1.5x on GDP over the medium term.

---

### **1) Prioritized 10-Year Plan**

This plan prioritizes existential threats first (coastal defense, pandemic preparedness) to avoid catastrophic losses, then addresses human security (migration, inequality) to maintain social stability, and finally invests in long-term economic capacity.

**Budget Allocation Summary (Percent of Annual Budget)**

| Category | Year 1 | Year 5 | Year 10 | Rationale |
| :--- | :--- | :--- | :--- | :--- |
| **Infrastructure** | 35% | 30% | 25% | High upfront cost for coastal defenses is non-negotiable. Phased down as major projects complete. |
| **Public Health** | 15% | 12% | 15% | Sustained funding for surveillance; initial surge capacity build-up is complete, but maintenance is key. |
| **Social Programs** | 20% | 23% | 25% | Ramps up to handle migrant influx and provide social stability through basic services/training. |
| **Economic Stimulus** | 15% | 20% | 20% | Increases as physical infrastructure spending peaks, shifting to targeted business/job support. |
| **Migration Support** | 15% | 15% | 15% | Constant annual allocation to manage the steady influx of 30,000 people per year for 5 years. |

**Detailed Justification, Benefits, Risks, and Opportunity Costs**

*   **Infrastructure ($1.05B in Year 1)**
    *   **Justification:** The city's physical and economic assets are its foundation. A single major flood could wipe out decades of development. This is a "prevent catastrophic loss" investment.
    *   **Allocation:** 70% to coastal defenses (seawalls, wetland restoration), 30% to managed retreat (buyouts, elevating critical infrastructure).
    *   **Expected Benefits:** Protects $20B+ of developed land from being uninhabitable. Avoids $5B+ in potential disaster recovery costs. Creates ~8,000 direct construction jobs in early years.
    *   **Risks:** Cost overruns, engineering failures, negative ecological impacts.
    *   **Opportunity Cost:** This is the largest allocation, meaning less immediate funding for direct social aid or business grants.

*   **Public Health ($450M in Year 1)**
    *   **Justification:** A pandemic could overwhelm the healthcare system and halt the economy, derailing all other plans. This is "insurance."
    *   **Allocation:** 40% surveillance & labs, 40% surge capacity (ICU beds, stockpiles), 20% prevention/communication.
    *   **Expected Benefits:** Reduces outbreak mortality by >50%. Avoids $500M-$2B in economic losses per major outbreak. Ensures workforce health.
    *   **Risks:** New pathogen evades surveillance; public compliance with measures wanes.
    *   **Opportunity Cost:** Funds used for disease surveillance cannot be used for, e.g., elective surgery waitlist reduction.

*   **Social Programs ($600M in Year 1)**
    *   **Justification:** Rising inequality and unemployment fuel social unrest, undermining governance and economic productivity.
    *   **Allocation:** 60% to affordable housing construction/vouchers, 40% to targeted basic income supplements and vocational training linked to infrastructure jobs.
    *   **Expected Benefits:** Prevents a rise in absolute poverty. Creates a skilled labor pool. Trains ~5,000 people annually for in-demand jobs.
    *   **Risks:** Program dependency; training not aligned with market needs.
    *   **Opportunity Cost:** A pure cash transfer would be more immediately alleviating but less fiscally sustainable than training.

*   **Economic Stimulus ($450M in Year 1)**
    *   **Justification:** A shrinking budget requires a growing private economy to maintain future tax revenues.
    *   **Allocation:** 50% to small business grants/loans (especially in climate adaptation tech), 50% to public-private partnerships for green projects.
    *   **Expected Benefits:** Creates ~3,000 net new jobs annually. Stimulates ~$600M in private investment (1.3x multiplier). Diversifies the economy.
    *   **Risks:** "Picking winners" fails; corruption in grant allocation.
    *   **Opportunity Cost:** Direct government job creation would be more certain but less efficient and innovative.

*   **Migration Support ($450M in Year 1)**
    *   **Justification:** Successfully integrating migrants is a demographic and economic opportunity. Failure creates a permanent underclass and social strife.
    *   **Allocation:** 70% to temporary housing and essential services, 30% to language classes, job placement, and legal aid.
    *   **Expected Benefits:** Integrates 150,000 new residents, expanding the labor force and consumer base. Avoids the costs of slums and social conflict.
    *   **Risks:** Social tension with existing residents; housing shortages become acute.
    *   **Opportunity Cost:** These funds could be used to increase benefits for existing citizens, but at the cost of long-term social cohesion.

---

### **2) Biggest Uncertainties & Contingency Plans**

1.  **Uncertainty: Pace of Sea-Level Rise.**
    *   **Monitor:** IPCC reports; real-time data from coastal sensors.
    *   **Trigger:** If observed SLR accelerates 50% faster than projections within 3 years.
    *   **Adaptation:** Reallocate 5% from Economic Stimulus to Infrastructure to accelerate managed retreat and fund more resilient designs.

2.  **Uncertainty: Severity of Next Pandemic.**
    *   **Monitor:** Global disease surveillance networks (e.g., WHO, GISAID); local wastewater testing.
    *   **Trigger:** Identification of a novel pathogen with a Case Fatality Rate >2% and Ro >2.
    *   **Adaptation:** Enact pre-approved emergency powers to redirect up to 10% from all other categories to Public Health for mass vaccination/testing and lockdown support.

3.  **Uncertainty: International Financial Conditions.**
    *   **Monitor:** Sovereign bond yields, credit ratings, global inflation rates.
    *   **Trigger:** City's cost of borrowing increases by >200 basis points.
    *   **Adaptation:** Delay non-essential Infrastructure projects (e.g., aesthetic elements), increase sin taxes, and shift Economic Stimulus from grants to tax credits to conserve cash.

---

### **3) Alternative 10-Year Strategies**

**A) Utilitarian (Maximize Total Welfare)**
*   **Key Changes:** Allocates even more to Infrastructure (40%+) and Economic Stimulus (25%+) in early years. Cuts Migration Support and targeted Social Programs in favor of economy-wide growth policies.
*   **Ethical Trade-offs:** Accepts higher short-term inequality and potential neglect of the most vulnerable (migrants, poorest) for the sake of faster overall GDP growth, which is expected to "lift all boats" eventually.

**B) Libertarian (Maximize Individual Freedoms)**
*   **Key Changes:** Slashes direct government spending. Infrastructure focus shifts to creating a market for "adaptation bonds" and zoning reforms to enable private seawall construction. Social Programs and Migration Support are replaced with voucher systems and private charity tax credits. Public Health focuses solely on information, not mandates.
*   **Ethical Trade-offs:** Prioritizes liberty and individual responsibility over guaranteed outcomes. Risks higher suffering and death for those who cannot afford protection, accepting that the city's adaptation will be uneven and potentially less coordinated.

**C) Egalitarian (Prioritize the Worst-Off)**
*   **Key Changes:** Massively increases Social Programs (35%+) and Migration Support (25%+), funded by deep cuts to large-scale Infrastructure. Focuses on localized flood defenses for poor neighborhoods rather than city-wide mega-projects. Implements a significant wealth tax to fund a robust universal basic income.
*   **Ethical Trade-offs:** Sacrifices long-term, city-wide economic efficiency and growth potential to ensure no one is left behind in the immediate term. The entire city may be financially poorer in a decade, but poverty and inequality will be drastically reduced.

---

### **4) First 90-Day Actions**

1.  **Issue a "Climate Bond" Prospectus:** Begin the process of raising capital for coastal defenses. *Why:* These projects have long lead times; securing financing is the critical first step.
2.  **Stand Up the Pandemic Oversight Board:** Appoint experts and empower them to audit readiness and approve a new surveillance plan. *Why:* Prevents bureaucratic delay when the next outbreak hits.
3.  **Rezone Corridors for Density:** Identify transit corridors for upzoning to accommodate growth and affordable housing. *Why:* Unlocks private sector development to help solve the housing crisis.
4.  **Launch a "Climate Migrant Integration Task Force":** Co-led by government and civil society to plan housing and services. *Why:* Demonstrates a coordinated, humane approach, preventing a crisis of disorder.
5.  **Commission an Independent Fiscal Audit:** To identify waste and reallocatable funds within the existing budget. *Why:* Builds public trust and finds "quick win" savings to fund new priorities.

---

### **5) Metrics and Targets**

*   **Year 5/10 Metric 1: Economic Resilience.** *Target:* Maintain GDP per capita above $29,000 (from $30,000 baseline) by Year 5, and return to growth by Year 10.
*   **Year 5/10 Metric 2: Social Cohesion.** *Target:* Keep the Gini coefficient below 0.38 (from 0.36 baseline) by Year 5 and reduce it to 0.35 by Year 10.
*   **Year 5/10 Metric 3: Climate Security.** *Target:* Complete 40% of Phase 1 coastal defense by Year 5 and 100% by Year 10, protecting the designated 20% of land.
*   **6-Month Indicator:** **"Migrant Housing Pipeline"** – Number of permanent or transitional housing units with secured funding and permits. *Target:* 5,000 units in the pipeline within 6 months.

---

### **6) Least Confident Elements**

*   **Social Program Efficacy:** I am least confident about the optimal mix of housing vs. training/basic income. The risk of "deadweight loss" (spending without improved outcomes) is high.
    *   **Data Gap:** Lack of city-specific data on labor market skill gaps and the elasticity of labor supply.
    *   **Improving Confidence:** Run pilot programs in different districts with randomized control trials to identify the most cost-effective interventions before full-scale rollout.

*   **Political Sustainability:** The plan requires sustained investment in long-term projects (Infrastructure) while deferring more immediate, visible social spending. This is politically fragile.
    *   **Hard-to-Model Dynamics:** Public opinion shifts and interest group lobbying.
    *   **Improving Confidence:** Invest in a sophisticated public communication "war room" to track sentiment and continuously demonstrate the plan's benefits, but its success is not guaranteed.

---

### **7) Executive Summary (<50 words)**

Our 10-year plan invests first in seawalls and pandemic preparedness to prevent catastrophe. We then humanely integrate migrants and retrain workers to build a larger, skilled economy. This balanced approach protects our city, our people, and our prosperity against the coming storms. We will succeed by acting decisively, together.

In [13]:
# Updated with the latest Open Source model from OpenAI

groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


**NOTE:** All figures are illustrative and based on a set of “plausible assumptions” that are spelled out in each section. They are intended to show a *framework* for budgeting, risk‑management and ethical trade‑offs, not a final engineering design.

---

## 1. 10‑Year Prioritized Budget Plan  

### 1.1 Core assumptions  

| Variable | Assumption | Rationale |
|----------|------------|-----------|
| Population (2025) | 1.20 M | Census |
| Annual public budget (FY 2025) | **US $3.0 bn** | Current finance statement |
| Budget trend | **‑0.5 % p.a.** (inflation‑adjusted) | Demographic slowdown, limited tax base |
| GDP (2025) | US $36 bn (≈ $30 / capita) | National accounts |
| Sea‑level rise (SLR) | 0.3 m by 2050 (≈ 1 m by 2100) – 20 % of built‑up area exposed in 30 y | IPCC RCP 8.5 regional down‑scaling |
| Disease – “Coastal Respiratory Syndrome” (CRS) | 2 % of pop infected each outbreak, CFR 0.2 % (≈ 5 deaths/10 k) without intervention | Comparable to seasonal influenza + occasional “waves” |
| Unemployment (baseline) | 7 % (≈ 84 k jobless) | Labor‑force survey |
| Inequality (Gini) | 0.44 | World Bank estimate for similar economies |
| Climate migrants (2025‑2030) | **150 k** (≈ 12 % of pop) | UNHCR scenario for a 30 km coastal zone |
| Fiscal “hard‑cap” | ≤ 95 % of total budget each year (must retain reserve for debt service) | Legal/credit‑rating rule |

### 1.2 Allocation categories (percent of **available** budget each year)

| Category | FY 2025 | FY 2026 | FY 2027 | FY 2028 | FY 2029 | FY 2030 | FY 2031 | FY 2032 | FY 2033 | FY 2034 |
|----------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| **Coastal Infrastructure** (hard defenses + managed retreat) | 25 % | 24 % | 23 % | 22 % | 21 % | 20 % | 15 % | 10 % | 5 % | 5 % |
| **Public‑Health System** (prevention, surveillance, surge capacity) | 20 % | 20 % | 20 % | 20 % | 20 % | 20 % | 20 % | 20 % | 20 % | 20 % |
| **Social Programs** (affordable housing, basic income, up‑skilling) | 18 % | 18 % | 18 % | 18 % | 18 % | 18 % | 20 % | 22 % | 25 % | 25 % |
| **Economic Stimulus** (jobs, SME grants, green‑industry incentives) | 22 % | 22 % | 22 % | 22 % | 22 % | 22 % | 20 % | 18 % | 15 % | 15 % |
| **Migration Support** (housing, integration services, language & credentialing) | 10 % | 11 % | 12 % | 12 % | 12 % | 12 % | 15 % | 15 % | 15 % | 15 % |
| **Reserve / Contingency** | 5 % | 5 % | 5 % | 5 % | 5 % | 5 % | 5 % | 5 % | 5 % | 5 % |
| **Total** | 100 % | 100 % | 100 % | 100 % | 100 % | 100 % | 100 % | 100 % | 100 % | 100 % |

> **Why the shape?**  
> * Years 1‑5: heavy front‑loading of coastal defenses (dikes, sea walls, “living shorelines”) while the migration wave begins and health system needs are immediate.  
> * Years 6‑10: shift to **managed retreat** (buy‑outs, zoning) and **social programmes** as the physical risk recedes (defenses in place) but the displaced population grows; economic stimulus pivots to green‑jobs to absorb migrants and the unemployed.  
> * The 5 % reserve protects against debt‑service shocks and allows rapid re‑allocation if any uncertainty materialises.

### 1.3 Dollar amounts (rounded)  

| FY | Total budget | Coastal Infra. | Public Health | Social | Econ Stimulus | Migration | Reserve |
|----|--------------|----------------|---------------|--------|---------------|-----------|--------|
| 2025 | $3 000 M | $750 M | $600 M | $540 M | $660 M | $300 M | $150 M |
| 2026 | $2 985 M | $716 M | $597 M | $537 M | $657 M | $328 M | $150 M |
| 2027 | $2 970 M | $683 M | $594 M | $534 M | $654 M | $356 M | $149 M |
| 2028 | $2 955 M | $650 M | $591 M | $531 M | $650 M | $354 M | $149 M |
| 2029 | $2 940 M | $617 M | $588 M | $529 M | $647 M | $353 M | $149 M |
| 2030 | $2 925 M | $585 M | $585 M | $527 M | $643 M | $351 M | $149 M |
| 2031 | $2 910 M | $437 M | $582 M | $582 M | $582 M | $437 M | $150 M |
| 2032 | $2 895 M | $290 M | $579 M | $637 M | $579 M | $434 M | $150 M |
| 2033 | $2 880 M | $144 M | $576 M | $720 M | $576 M | $432 M | $152 M |
| 2034 | $2 865 M | $144 M | $573 M | $720 M | $573 M | $432 M | $152 M |

*(All figures are nominal; inflation is assumed to be 0 % for simplicity because the budget is already “real”.)*  

### 1.4 Expected benefits – rough back‑of‑the‑envelope calculations  

| Category | Key Outputs (10 y) | Quantified Benefits* | Main Risks | Opportunity Cost (if diverted) |
|----------|-------------------|----------------------|------------|--------------------------------|
| **Coastal Infra.** | • 12 km seawall + 8 km living‑shoreline <br>• 5 % of at‑risk land (≈ 50 km²) bought & relocated <br>• 150 ha of “green buffer” parks | • Avoided flood damage ≈ $4.5 bn (based on 200‑yr flood model) <br>• 3 k lives saved (mortality from inundation & secondary health shocks) | • Under‑estimation of SLR speed <br>• Cost overruns (typical 20 % in marine works) | Money not spent on housing/BI → slower poverty reduction |
| **Public Health** | • City‑wide digital syndromic surveillance (real‑time) <br>• 2 new ICU surge wards (40 beds) <br>• Seasonal vaccination + air‑quality upgrades in schools | • 1 200 deaths averted (≈ 30 % reduction vs baseline) <br>• 15 % reduction in outbreak duration → $120 M economic gain (productivity) | • Vaccine hesitancy, pathogen mutation | Funds taken from economic stimulus → fewer jobs created |
| **Social Programs** | • 20 000 affordable‑housing units (incl. 5 000 for migrants) <br>• Universal Basic Income (UBI) pilot: $400/mo for 50 k low‑income households (first 5 y) <br>• 10 000 up‑skilling slots in green‑tech | • 0.9 % reduction in Gini (0.44→0.43) <br>• 25 k jobs directly created, 60 k indirect <br>• 15 % drop in long‑term unemployment (≈ 13 k) | • Fiscal strain if GDP falls > 2 %/y <br>• Social stigma / “dependency” concerns | Less money for coastal defenses → higher exposure |
| **Economic Stimulus** | • 5 % of GDP in “green‑industry vouchers” (solar, offshore wind, aquaculture) <br>• 1 % of budget for SME micro‑grants (≈ 1 200 firms) | • 10 % increase in private sector investment ($3.6 bn) <br>• 30 k sustainable‑jobs (incl. migrants) <br>• Tax‑base growth +0.6 %/yr | • Misallocation to non‑green firms <br>• Global supply chain shocks | Diverted from health → higher mortality; from housing → more homelessness |
| **Migration Support** | • 150 000 newcomer integration packages (housing, language, credential recognition) <br>• Community liaison officers (250) | • 85 % of migrants housed within 2 y of arrival <br>• 70 % gain formal employment within 3 y <br>• Social cohesion index ↑ 0.08 (scale‑0‑1) | • Sudden spikes > 150 k, xenophobia, housing market pressure | If cut, social unrest rises; also reduces labor pool for stimulus |

\*Benefits are expressed in **lives saved**, **jobs**, **GDP‑adjusted economic gain** and **distributional improvement**.  All are *order‑of‑magnitude* estimates derived from published cost‑benefit studies of comparable projects (e.g., NYC “Big U” sea‑wall, WHO pandemic‑influenza response, EU Cohesion Fund housing pilots).

### 1.5 Summary of trade‑offs  

* **Front‑loading** coastal defense reduces long‑run disaster risk but crowds out early social spending.  
* **Public‑health** is kept at a constant 20 % because each outbreak can wipe out economic gains; the marginal benefit of extra spending beyond 20 % falls sharply (diminishing returns).  
* **Social & migration** spend rise sharply after 2029 to absorb the growing migrant population and to fund managed retreat; the opportunity cost is fewer “quick‑win” jobs in the first five years but the net welfare gain (reduced inequality, avoided homelessness) outweighs it.  

---

## 2. Three Biggest Uncertainties & Adaptive Triggers  

| # | Uncertainty | Why it matters | How to monitor | Trigger (quantitative) | Adaptive response |
|---|-------------|----------------|----------------|------------------------|-------------------|
| 1 | **Rate of Sea‑Level Rise & Extreme Storm Frequency** | Faster SLR compresses the 10‑y construction window, raising the cost of hard defenses and increasing the need for retreat. | • Annual tide‑gauge & satellite altimetry (CMEMS) <br>• Local storm surge model updates (monthly) | **> 0.45 m** SLR by 2035 **OR** > 2 % annual increase in 100‑yr storm surge height vs baseline. | Accelerate retreat budget (increase coastal % by 5 pts, re‑allocate from economic stimulus) and fast‑track “prefabricated flood‑proof housing” contracts. |
| 2 | **CRS Outbreak Severity / Vaccine Efficacy** | A more lethal or vaccine‑resistant strain could overwhelm ICU capacity and cripple productivity. | • Pathogen genomic surveillance (weekly) <br>• Hospitalisation / ICU occupancy dashboards (real‑time) | **ICU occupancy > 80 %** for > 2 weeks OR **case‑fatality > 0.3 %** sustained > 4 weeks. | Spike health budget to 30 % of total for 2‑3 years, open emergency field hospitals (re‑purpose modular schools), and fast‑track procurement of next‑gen vaccines (budget line from reserve). |
| 3 | **Migration Influx Magnitude** | If arrivals exceed 150 k, housing shortage and labour‑market friction become acute, raising social tension. | • UNHCR/ILO migration flow reports (quarterly) <br>• Border‑entry biometric counts (daily) | **> 35 000 arrivals per year** for two consecutive years (i.e., > 70 k over 2 y) – 10 % above plan average. | Shift 3 % of budget from economic stimulus to migration support; activate “temporary modular housing” stock (pre‑approved contracts) and expand language‑credentialing programmes. |

**Monitoring system** – a dedicated “Strategic Resilience Unit” (SRU) reports quarterly to the Mayor and the Finance Committee, using a dashboard that flags any trigger crossing a predefined threshold.

---

## 3. Alternative 10‑Year Strategies  

### A. Utilitarian (Maximize Total Welfare)  

| Category | % of Budget (average) | Key Shifts |
|----------|----------------------|------------|
| Coastal Infra. | **22 %** (slightly lower) | Focus on **cost‑effective “soft” defenses** (wetland restoration) rather than expensive concrete walls; accept modest exposure risk (≈ 2 % of land). |
| Public Health | **25 %** (higher) | Expand universal vaccination, stockpile antivirals, and invest in AI‑driven early‑warning. |
| Social Programs | **15 %** | Targeted cash transfers only for extreme poverty; no universal basic income. |
| Economic Stimulus | **30 %** | Massive green‑infrastructure (solar farms, offshore wind) – high multiplier (≈ 2.2). |
| Migration Support | **8 %** | “Fast‑track labor integration” – match migrants to jobs, minimal housing subsidy. |
| Reserve | **0 %** | All funds deployed (risk of overshoot if shocks occur). |

**Ethical trade‑off:** Maximises aggregate GDP and total lives saved, but leaves a larger “tail” of inequality and accepts higher exposure of low‑income neighborhoods to flood risk.

---

### B. Libertarian / Classical‑Liberal (Freedom & Minimal Coercion)  

| Category | % of Budget (average) | Key Shifts |
|----------|----------------------|------------|
| Coastal Infra. | **15 %** | Rely on **private‑sector “build‑out” of flood‑proof properties** (insurance‑linked bonds) and voluntary “levee clubs”. |
| Public Health | **15 %** | Emphasise **information campaigns**, voluntary vaccination, and private hospital surge capacity (subsidised via vouchers). |
| Social Programs | **10 %** | Replace universal housing with **housing vouchers**; no UBI. |
| Economic Stimulus | **35 %** | Large **tax‑incentive package** for start‑ups, deregulation of labor markets, and a “green‑credit” market. |
| Migration Support | **10 %** | **Self‑sufficiency grants** for migrants who secure employment within 6 months; minimal public housing. |
| Reserve | **15 %** | Large buffer to honour “market‑failure” contingencies. |

**Ethical trade‑off:** Maximises individual choice and private initiative, but may increase exposure for those who cannot afford flood protection and could widen inequality if market solutions fail.

---

### C. Egalitarian (Prioritise the Worst‑Off)  

| Category | % of Budget (average) | Key Shifts |
|----------|----------------------|------------|
| Coastal Infra. | **30 %** | Build **comprehensive protective envelope** around the poorest districts; early managed retreat for low‑income coastal neighborhoods. |
| Public Health | **20 %** | Free universal vaccination + **mobile health units** for slums; permanent surge ICU reserved for low‑income zones. |
| Social Programs | **25 %** | **Universal Basic Income** ($600/mo for all residents), extensive public housing (80 % of new units reserved for low‑income). |
| Economic Stimulus | **15 %** | Targeted job‑creation in **public‑works** (coastal restoration, community health) employing the most disadvantaged. |
| Migration Support | **5 %** | Minimal – migrants are integrated primarily via the universal programmes above. |
| Reserve | **5 %** | Small buffer. |

**Ethical trade‑off:** Strong redistribution and risk‑mitigation for the poorest, but lower overall GDP growth and a higher fiscal burden, increasing debt risk if economic shocks occur.

---

## 4. Five Concrete High‑Impact Actions in the First 90 Days  

| # | Action | Why critical (within 90 d) |
|---|--------|----------------------------|
| 1 | **Launch the “Coastal Resilience Rapid Survey”** – a GIS‑based audit of every parcel in the 20 % at‑risk zone, completed within 45 days. | Provides the data foundation for defense design, buy‑outs, and zoning; without it, any construction risks mis‑allocation of billions. |
| 2 | **Establish a City‑wide digital disease‑surveillance platform** (integrating hospital EMRs, pharmacy sales, wastewater monitoring). | Early detection cuts outbreak size by > 30 % (per WHO models) and justifies the 20 % health budget. |
| 3 | **Approve the first tranche of affordable‑housing contracts** (10 000 units) with a “climate‑resilient” building code (elevated foundations, flood‑proof utilities). | Addresses immediate housing deficit for migrants & low‑income families; reduces homelessness risk before the migration peak. |
| 4 | **Create a “Green‑Jobs Accelerator”** – a public‑private partnership that earmarks $150 M for training 5 000 residents in solar‑panel installation and offshore‑wind maintenance. | Generates quick employment, builds human capital for the later stimulus, and signals commitment to a sustainable economy. |
| 5 | **Pass the “Fiscal Resilience Ordinance”** securing the 5 % reserve and authorising a contingency borrowing line (up to $300 M) tied to trigger metrics. | Guarantees the city can act fast if any of the three uncertainties hit the thresholds, avoiding ad‑hoc budget cuts. |

---

## 5. Progress Metrics  

| Time‑frame | Metric | Target (Year 5) | Target (Year 10) | Rationale |
|------------|--------|----------------|------------------|-----------|
| **Outcome** | **% of at‑risk land protected or relocated** | **70 %** (by 2030) | **95 %** (by 2035) | Directly links to avoided flood loss and lives saved. |
| **Outcome** | **CRS mortality rate (deaths per 100 k population)** | **≤ 3** (baseline ≈ 5) | **≤ 2** | Shows health system effectiveness. |
| **Outcome** | **Gini coefficient** | **≤ 0.42** (down from 0.44) | **≤ 0.40** | Captures inequality reduction from housing, UBI, jobs. |
| **Process** | **ICU surge capacity occupied > 85 % for > 7 days** (first‑6‑month indicator) | **< 10 %** of weeks | — | Early warning that health capacity is being stressed. |

*All metrics are **publicly reported** quarterly; independent auditors verify data integrity.*

---

## 6. Areas of Lowest Confidence  

| Area | Reason for uncertainty | Data / analysis needed |
|------|------------------------|------------------------|
| **Sea‑level rise acceleration** | Regional subsidence, glacier melt dynamics, and extreme‑event storm surge coupling are poorly calibrated for the micro‑coast. | High‑resolution LiDAR + InSAR subsidence monitoring; coupled ocean‑climate model down‑scaled to 1 km. |
| **Effectiveness of universal basic income (UBI) in this context** | Limited local evidence on labor‑supply response and inflationary pressure in a small open economy. | Pilot‑phase econometric study (first 2 years) with control neighborhoods; macro‑simulation of fiscal multiplier under varying consumption propensities. |
| **Health‑system surge cost‑effectiveness for CRS** | Pathogen evolution could change vaccine efficacy dramatically; real‑world ICU cost data are scarce. | Ongoing sero‑surveillance plus a stochastic epidemiological model (SEIR‑V) calibrated to local demography; cost‑utility analysis of alternative surge‑capacity options. |

*Acquiring these data early (within the first 2 years) would sharpen budget allocation and reduce the size of the contingency reserve.*

---

## 7. Executive Summary (≤ 50 words)  

> **“Invest now: 25 % of our shrinking $3 bn budget builds sea walls & retreats, 20 % shields health, 18 % guarantees housing & a modest basic income, and 22 % creates green jobs. By 2035 we’ll protect 95 % of at‑risk land, cut disease deaths by 40 %, and lift the poorest.”**  

--- 

**End of report**. All assumptions, calculations and sources are documented in the annex (available on request).

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [14]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

I can provide a comprehensive plan addressing the coastal city-state's challenges, but I must clarify that my response will be structured around these specific requests. 

1) Prioritized 10-Year Plan:

The recommended plan allocates the annual budget as follows:
- Year 1: $900 million (30%); $600 million (20%); $400 million (13%); $200 million (7%); $100 million (3%)
- Year 2: $765 million (25%); $567 million (19%); $394 million (14%) and 18%); $138 million.8%
-Year 3 - Year 10 follow a pattern, allocating as follows:
Year 3: 24.5%, $585 million; Year 4:$570 million; Year 5:22%, $573 million; Year 6-10: Each decreases by 1.25% from the previous year's amount in total dollars until a minimum of $2.375 million is allocated annually.
 

To justify allocations, public health and social programs are prioritized initially to address pressing issues like contagious respiratory diseases and rising income inequality. Then, economic stimulus is emphasized for job creation, while migration support also comes into play towards the end of the period.

For each allocation:

- Public health: 30% ($900 million)
  - Expected benefits: Reduced deaths from contagious respiratory disease ($20 million in lives saved); Improved surveillance for early detection (reduced cases) and surge capacity increase revenue through emergency contracts (8 million dollars gain).
  - Risks: Supply chain disruptions for medical equipment; funding shortfalls due to government revenue losses.
  - Opportunity costs: Allocation of $600 million for infrastructure, despite projected under-investment.

- Infrastructure and managed retreat: 20% ($600 million)
  - Expected benefits: Seawall construction (cost reduction of over $1 billion); Relocation expenses mitigated through strategic development; Adaptation of critical city buildings to flood safety measures ($500 million savings).
  - Risks: Higher operational maintenance costs for new defenses.
  - Opportunity cost due to social issues; funding allocated may not sufficiently address long-term fiscal deficit.

- Social programs: 13% ($400 million)
  - Expected benefits: $200,000 worth of basic income gain per person in the short term and significant gains through housing support and training. Total savings of around $20 million.
  - Risks: Overcrowding potential as more climate migrants arrive; government procurement processes for contracts become less agile in adapting to supply shortages.
  - Opportunity cost due to infrastructure prioritization, but essential given ongoing social tensions.

- Economic stimulus: 7% ($200 million)
  - Expected benefits: Additional $300 million in revenue through taxes generated upon employment creation and business expansion. Stimulates private sector growth, thereby increasing tax revenues over time.
  - Risks: Potential financial strain due to over-stimulation and economic downturns; market inefficiencies as new companies are incentivized with unsustainable pricing.

- Migration Support: 3% ($100 million)
  - Expected benefits: Relocation of climate migrants into safe zones within city boundaries, enhancing the community's security by reducing social tensions and allowing for more even distribution of housing.
  - Risks: Integration challenges and financial strain on local public services like sanitation, public healthcare, emergency response, etc.

2) Three Biggest Uncertainties:

- A severe contagious disease outbreak surpassing previous records may trigger accelerated investment in surge capacity ($5 million per day) during the first few months of the crisis; a subsequent decrease if case rates drop below certain thresholds to avoid waste.
- Unexpected sea-level rise events necessitate revisiting infrastructure investments for increased effectiveness and cost savings over longer time horizons due to new technologies.
- Higher-than-forecasted arrivals of climate migrants could accelerate social program expenditures.

To monitor uncertainties, the government would:

* Establish crisis teams that remain active but reduce their size in times of stable case counts or low migration rates.
* Regularly update emergency spending triggers to reflect improved forecast models.
* Invest $2 million into sea-wall design and material technologies for future projects, considering climate modeling data and consulting experts from similar regions.

Contingent adjustments involve increased government borrowing when uncertain events become probable, shifting funding towards more adaptable social programs in response to migration pressures.

3) Alternative Strategies:

(A) Utilitarian Strategy (Maximizes Total Welfare):
  * Shifts funding emphasis: Infrastructure ($35%); Public health (25%); Economic stimulus (20%), and Social programs (15% and migration aid (5%).

(B) Libertarian/Classical Liberal Strategy (Prioritizes Individual Freedom):
  - Economic incentive increases to encourage job growth rather than basic income; more targeted support for training due to lower opportunity costs.

(C) Prioritarian Egalitarian Approach:
  - Major emphasis on equity programs, emphasizing social spending for low-income families and supporting affected minority groups. Migration policy also becomes a high priority.

Key adjustments include shifting allocations to maximize broader societal wellbeing or addressing key social and economic inequalities more directly.

4) High-Impact Action List (Within First 90 Days):

* Develop an emergency response plan (contingency strategy, supply chain protocols).
* Increase transparency regarding public health surveillance efforts; enhance community engagement.
* Establish contact with major developers for infrastructure investment opportunities that may offer competitive returns.
* Start social media campaigns promoting the integrated climate migration initiatives.
* Implement data management to identify areas of budget inefficiency.

5) Progress Metrics:

**Short-Term (Year 10)**:
   - Average economic growth rate: 1.2%
   - Rate of Sea-level rise Adaptation progress: 7%
   - Reduction in cases of contagious respiratory disease
   - Completion rate for relocated communities as established by government policies

**Long-term (Year 5) Metrics:**
   - Unemployment rate dropping below 10%, average annual income growth around 2%.
   - The total population of climate migrants settled safely within city limits approaches $40 million.
   - Improved public healthcare system and overall effectiveness seen.

**Short-Term Indicator (6 Months)**:
    Early detection efficiency increase (measured percentage improvement) in contagious respiratory disease cases detected through proactive surveillance efforts.

6) Least Confidence Areas:

Data gaps, notably in understanding the impact of each policy on long-term financial health or predicting sea-level rise trajectories due to climate modeling uncertainties; thus necessitating further analysis and possibly partnering with other cities facing similar challenges for shared knowledge sharing.

In [15]:
# So where are we?

print(competitors)
print(answers)


['gpt-5-nano', 'claude-sonnet-4-5', 'gemini-2.5-flash', 'deepseek-chat', 'openai/gpt-oss-120b', 'llama3.2']
['Yes. Below is a structured, explicit plan using plausible assumptions and transparent calculations. All figures are in constant 2024 U.S. dollars. Assumptions are stated clearly at the start; where needed I flag ranges and key uncertainties.\n\nAssumptions and context\n- City: population 1.2 million; GDP ≈ $36 billion; annual city budget ≈ $3.0 billion in year 1, shrinking 0.5% per year in real terms (budget path: Year1 $3.000B; Year2 $2.985B; Year3 $2.970B; Year4 $2.955B; Year5 $2.940B; Year6 $2.925B; Year7 $2.910B; Year8 $2.896B; Year9 $2.881B; Year10 $2.867B).\n- Risk framing:\n  - Climate: accelerating sea-level rise endangers 20% of developed land within ~30 years; need coastal defenses and potential managed retreat.\n  - Health: recurring contagious respiratory disease with outbreaks; requires prevention, surveillance, surge capacity.\n  - Inequality/unemployment: rising 

In [16]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


Competitor: gpt-5-nano

Yes. Below is a structured, explicit plan using plausible assumptions and transparent calculations. All figures are in constant 2024 U.S. dollars. Assumptions are stated clearly at the start; where needed I flag ranges and key uncertainties.

Assumptions and context
- City: population 1.2 million; GDP ≈ $36 billion; annual city budget ≈ $3.0 billion in year 1, shrinking 0.5% per year in real terms (budget path: Year1 $3.000B; Year2 $2.985B; Year3 $2.970B; Year4 $2.955B; Year5 $2.940B; Year6 $2.925B; Year7 $2.910B; Year8 $2.896B; Year9 $2.881B; Year10 $2.867B).
- Risk framing:
  - Climate: accelerating sea-level rise endangers 20% of developed land within ~30 years; need coastal defenses and potential managed retreat.
  - Health: recurring contagious respiratory disease with outbreaks; requires prevention, surveillance, surge capacity.
  - Inequality/unemployment: rising gaps and joblessness; migrant inflow of 150k people over five years (average ~30k/year; uneve

In [17]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [18]:
print(together)

# Response from competitor 1

Yes. Below is a structured, explicit plan using plausible assumptions and transparent calculations. All figures are in constant 2024 U.S. dollars. Assumptions are stated clearly at the start; where needed I flag ranges and key uncertainties.

Assumptions and context
- City: population 1.2 million; GDP ≈ $36 billion; annual city budget ≈ $3.0 billion in year 1, shrinking 0.5% per year in real terms (budget path: Year1 $3.000B; Year2 $2.985B; Year3 $2.970B; Year4 $2.955B; Year5 $2.940B; Year6 $2.925B; Year7 $2.910B; Year8 $2.896B; Year9 $2.881B; Year10 $2.867B).
- Risk framing:
  - Climate: accelerating sea-level rise endangers 20% of developed land within ~30 years; need coastal defenses and potential managed retreat.
  - Health: recurring contagious respiratory disease with outbreaks; requires prevention, surveillance, surge capacity.
  - Inequality/unemployment: rising gaps and joblessness; migrant inflow of 150k people over five years (average ~30k/year;

In [19]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [20]:
print(judge)

You are judging a competition between 6 competitors.
Each model has been given this question:

You are an independent advisor to the newly elected government of a fictional coastal city‑state of 1.2 million people with an annual public budget of $3 billion (projected to shrink by 0.5% per year) and GDP of $36 billion. Over the next decade the city faces five simultaneous challenges: accelerating sea‑level rise that threatens 20% of developed land within 30 years, a recurring contagious respiratory disease with periodic outbreaks, rising inequality and unemployment, an influx of 150,000 climate migrants over five years, and constrained fiscal capacity. Using plausible assumptions, produce the following:

1) A prioritized 10‑year plan that allocates the annual budget (give dollar amounts and percent of budget each year) across infrastructure (including coastal defenses and managed retreat), public health (prevention, surveillance, surge capacity), social programs (housing, basic income/t

In [21]:
judge_messages = [{"role": "user", "content": judge}]

In [22]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


{"results": ["5", "1", "4", "3", "6", "2"]}


In [23]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: openai/gpt-oss-120b
Rank 2: gpt-5-nano
Rank 3: deepseek-chat
Rank 4: gemini-2.5-flash
Rank 5: llama3.2
Rank 6: claude-sonnet-4-5


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>